In [1]:
from dwapi import datawiz

from datetime import date, timedelta, datetime
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:.2f}'.format

In [2]:
dw = datawiz.DW()

# Первая табл - Общая по категориям

In [3]:
def clean_date(x):
    d = datetime.strptime(x, '%Y-%m-%d')
    new_d = d.strftime('%d-%m-%Y')
    return new_d


df = dw.get_categories_sale(date_from="2015-11-11", date_to="2015-11-18", by=['turnover','qty','receipts_qty'], view_type = 'raw')
df['date'] = df['date'].apply(lambda x: clean_date(x))

bla = pd.DataFrame(df.pivot_table(index = ['date'], values=['turnover','qty','receipts_qty'], aggfunc=sum))
bla['Середній чек'] = bla['turnover'] / bla['receipts_qty']
bla.index.names = ['Показник']
bla.rename(columns = {'turnover':'Оборот', 'qty':'Кількість товарів', 'receipts_qty':'Кількість чеків'}, inplace = True)

df = bla[['Оборот', 'Кількість товарів', 'Кількість чеків', 'Середній чек']].sort_index(ascending=0).T
del bla
df

Показник,18-11-2015,17-11-2015,16-11-2015,15-11-2015,14-11-2015,13-11-2015,12-11-2015,11-11-2015
Оборот,19823.23,48707.43,46346.55,45842.28,40801.53,49357.25,48524.38,48935.92
Кількість товарів,1775.81,4147.97,4073.08,3932.48,3532.79,4287.40,4253.21,4212.35
Кількість чеків,1403.00,3109.00,3212.00,2886.00,2618.00,3345.00,3336.00,3329.00
Середній чек,14.13,15.67,14.43,15.88,15.59,14.76,14.55,14.70


In [4]:
all_df = []

for i in range(len(df.columns)-1):
    try:
        df_small = df.iloc[:, i:i+2]
        df_small['Різниця'] = df.iloc[:,i] - df.iloc[:,i+2]
        df_small['Різниця в %'] = (df_small.loc[:, 'Різниця'] / df.iloc[:,i+2]).mul(100).round(2)
        
    except:
        df_small = df.iloc[:, i:]
        df_small['Різниця'] = df.iloc[:,i] - df.iloc[:,-1]
        df_small['Різниця в %'] = (df_small.loc[:, 'Різниця'] / df.iloc[:,-1]).mul(100).round(2)
        
    all_df.append(df_small)

In [5]:
for i in range(len(all_df)):
    print(all_df[i], '\n')

Показник           18-11-2015  17-11-2015   Різниця  Різниця в %
Оборот               19823.23    48707.43 -26523.32       -57.23
Кількість товарів     1775.81     4147.97  -2297.27       -56.40
Кількість чеків       1403.00     3109.00  -1809.00       -56.32
Середній чек            14.13       15.67     -0.30        -2.08 

Показник           17-11-2015  16-11-2015  Різниця  Різниця в %
Оборот               48707.43    46346.55  2865.15         6.25
Кількість товарів     4147.97     4073.08   215.49         5.48
Кількість чеків       3109.00     3212.00   223.00         7.73
Середній чек            15.67       14.43    -0.22        -1.37 

Показник           16-11-2015  15-11-2015  Різниця  Різниця в %
Оборот               46346.55    45842.28  5545.02        13.59
Кількість товарів     4073.08     3932.48   540.29        15.29
Кількість чеків       3212.00     2886.00   594.00        22.69
Середній чек            14.43       15.88    -1.16        -7.42 

Показник           15-11-2015

# Остальные табл - По продуктам

In [6]:
def clean_date(x):
    d = datetime.strptime(x, '%Y-%m-%d')
    new_d = d.strftime('%d-%m-%Y')
    return new_d


df = dw.get_products_sale(date_from="2015-11-11", date_to="2015-11-18", by=['turnover','qty'], view_type = 'raw')
df['date'] = df['date'].apply(lambda x: clean_date(x))

bla = df.groupby(['name', 'date']).sum().reset_index()
bla.drop(columns = ['product'], inplace = True)

# prepare df

qty = pd.pivot_table(bla, index = 'name', values = 'qty', columns = 'date', aggfunc = np.sum, fill_value=0)
turnover = pd.pivot_table(bla, index = 'name', values = 'turnover', columns = 'date', aggfunc = np.sum, fill_value=0)

qty = qty.T.sort_values('date', ascending = False).T
turnover = turnover.T.sort_values('date', ascending = False).T

In [7]:
negative_df = []
positive_df = []


for i in range(len(qty.columns)-1):
    try:        
        small_qty = qty.iloc[:, i:i+2]
        small_qty['Зміна кількості продаж'] = qty.iloc[:,i] - qty.iloc[:,i+2]
        
        small_turnover = turnover.iloc[:, i:i+2]
        small_turnover['Зміна обороту'] = turnover.iloc[:,i] - turnover.iloc[:,i+2]
        
        merged = pd.merge(small_qty, small_turnover, on = 'name')
        merged = merged[['Зміна кількості продаж','Зміна обороту']].reset_index().sort_values('Зміна обороту', ascending = True)
     
        
    except:
        small_qty = qty.iloc[:, i:]
        small_qty['Зміна кількості продаж'] = qty.iloc[:,i] - qty.iloc[:,-1]
        
        small_turnover = turnover.iloc[:, i:]
        small_turnover['Зміна обороту'] = turnover.iloc[:,i] - turnover.iloc[:,-1]
        
        merged = pd.merge(small_qty, small_turnover, on = 'name')
        merged = merged[['Зміна кількості продаж','Зміна обороту']].reset_index().sort_values('Зміна обороту', ascending = True)
    
#     merged['date_check'] = qty.columns[i]
    positive = merged[merged['Зміна обороту'] > 0].sort_values('Зміна обороту', ascending = False)
    negative = merged[merged['Зміна обороту'] < 0].sort_values('Зміна обороту', ascending = True)

    positive_df.append(positive)
    negative_df.append(negative)

In [8]:
for i in range(len(negative_df)):
    print(negative_df[i].head(2), '\n\n')

date                  name  Зміна кількості продаж  Зміна обороту
2283  STC Orelsky tin 700g                 -102.50        -752.35
1933    Parliament Silver                   -20.00        -546.00 


date                                        name  Зміна кількості продаж  \
2756       The mixture Nutrilon EAZY pack 1 800g                   -1.00   
808   Cognac of Ukraine 5* Chabot VSOP 40% 0.5 l                   -2.00   

date  Зміна обороту  
2756        -287.81  
808         -224.48   


date                                               name  \
1929                         Parliament Carat Sapphire    
1309  Flour Agrokhleb 1kg (write in multiples of 10kg)    

date  Зміна кількості продаж  Зміна обороту  
1929                  -11.00        -311.85  
1309                   -4.00        -223.96   


date                  name  Зміна кількості продаж  Зміна обороту
1521     L&M Loft Sea Blue                  -18.00        -296.10
2741  The coffee is normal                  -61.00

In [9]:
for i in range(len(positive_df)):
    print(positive_df[i].head(2), '\n\n')

date                                   name  Зміна кількості продаж  \
2757  The mixture Nutrilon EAZY pack 2 800g                    1.00   
162                      BAGUETTES in stock                   21.00   

date  Зміна обороту  
2757         307.66  
162          260.15   


date                    name  Зміна кількості продаж  Зміна обороту
1488  Kent HD NAVY BLUE (8)                    40.00         924.00
1927        Parliament Agua                    12.00         327.60 


date                              name  Зміна кількості продаж  Зміна обороту
2283              STC Orelsky tin 700g                   59.00         433.06
177   Banana GROCERY Diva (Ecuador) kg                    7.53         221.75 


date                                   name  Зміна кількості продаж  \
2756  The mixture Nutrilon EAZY pack 1 800g                    1.00   
1930                Parliament Carat Topaz                     9.00   

date  Зміна обороту  
2756         287.81  
1930         25